# <center> 【Kaggle】Telco Customer Churn 电信用户流失预测案例

---

## <font face="仿宋">第四部分导读

&emsp;&emsp;<font face="仿宋">在案例的第二、三部分中，我们详细介绍了关于特征工程的各项技术，特征工程技术按照大类来分可以分为数据预处理、特征衍生、特征筛选三部分，其中特征预处理的目的是为了将数据集整理、清洗到可以建模的程度，具体技术包括缺失值处理、异常值处理、数据重编码等，是建模之前必须对数据进行的处理和操作；而特征衍生和特征筛选则更像是一类优化手段，能够帮助模型突破当前数据集建模的效果上界。并且我们在第二部分完整详细的介绍机器学习可解释性模型的训练、优化和解释方法，也就是逻辑回归和决策树模型。并且此前我们也一直以这两种算法为主，来进行各个部分的模型测试。

&emsp;&emsp;<font face="仿宋">而第四部分，我们将开始介绍集成学习的训练和优化的实战技巧，尽管从可解释性角度来说，集成学习的可解释性并不如逻辑回归和决策树，但在大多数建模场景下，集成学习都将获得一个更好的预测结果，这也是目前效果优先的建模场景下最常使用的算法。

&emsp;&emsp;<font face="仿宋">总的来说，本部分内容只有一个目标，那就是借助各类优化方法，抵达每个主流集成学习的效果上界。换而言之，本部分我们将围绕单模优化策略展开详细的探讨，涉及到的具体集成学习包括随机森林、XGBoost、LightGBM、和CatBoost等目前最主流的集成学习算法，而具体的优化策略则包括超参数优化器的使用、特征衍生和筛选方法的使用、单模型自融合方法的使用，这些优化方法也是截至目前，提升单模效果最前沿、最有效、同时也是最复杂的方法。其中有很多较为艰深的理论，也有很多是经验之谈，但无论如何，我们希望能够围绕当前数据集，让每个集成学习算法优化到极限。值得注意的是，在这个过程中，我们会将此前介绍的特征衍生和特征筛选视作是一种模型优化方法，衍生和筛选的效果，一律以模型的最终结果来进行评定。而围绕集成学习进行海量特征衍生和筛选，也才是特征衍生和筛选技术能发挥巨大价值的主战场。

&emsp;&emsp;<font face="仿宋">而在抵达了单模的极限后，我们就会进入到下一阶段，也就是模型融合阶段。需要知道的是，只有单模的效果到达了极限，进一步的多模型融合、甚至多层融合，才是有意义的，才是有效果的。

---

# <center>Part 4.集成算法的训练与优化技巧

In [502]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

import warnings
warnings.filterwarnings('ignore')

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

# 自定义模块
from telcoFunc import *
# 导入特征衍生模块
import features_creation as fc
from features_creation import *

# re模块相关
import inspect, re

# 其他模块
from tqdm import tqdm
import gc

&emsp;&emsp;然后执行Part 1中的数据清洗相关工作：

In [503]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [504]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

&emsp;&emsp;同时，创建自然编码后的数据集以及经过时序特征衍生的数据集：

In [505]:
# 划分训练集和测试集
train, test = train_test_split(tcc, random_state=22)

X_train = train.drop(columns=[ID_col, target]).copy()
X_test = test.drop(columns=[ID_col, target]).copy()

y_train = train['Churn'].copy()
y_test = test['Churn'].copy()

X_train_seq = pd.DataFrame()
X_test_seq = pd.DataFrame()

# 年份衍生
X_train_seq['tenure_year'] = ((72 - X_train['tenure']) // 12) + 2014
X_test_seq['tenure_year'] = ((72 - X_test['tenure']) // 12) + 2014

# 月份衍生
X_train_seq['tenure_month'] = (72 - X_train['tenure']) % 12 + 1
X_test_seq['tenure_month'] = (72 - X_test['tenure']) % 12 + 1

# 季度衍生
X_train_seq['tenure_quarter'] = ((X_train_seq['tenure_month']-1) // 3) + 1
X_test_seq['tenure_quarter'] = ((X_test_seq['tenure_month']-1) // 3) + 1

# 独热编码
enc = preprocessing.OneHotEncoder()
enc.fit(X_train_seq)

seq_new = list(X_train_seq.columns)

# 创建带有列名称的独热编码之后的df
X_train_seq = pd.DataFrame(enc.transform(X_train_seq).toarray(), 
                           columns = cate_colName(enc, seq_new, drop=None))

X_test_seq = pd.DataFrame(enc.transform(X_test_seq).toarray(), 
                          columns = cate_colName(enc, seq_new, drop=None))

# 调整index
X_train_seq.index = X_train.index
X_test_seq.index = X_test.index

In [506]:
ord_enc = OrdinalEncoder()
ord_enc.fit(X_train[category_cols])

X_train_OE = pd.DataFrame(ord_enc.transform(X_train[category_cols]), columns=category_cols)
X_train_OE.index = X_train.index
X_train_OE = pd.concat([X_train_OE, X_train[numeric_cols]], axis=1)

X_test_OE = pd.DataFrame(ord_enc.transform(X_test[category_cols]), columns=category_cols)
X_test_OE.index = X_test.index
X_test_OE = pd.concat([X_test_OE, X_test[numeric_cols]], axis=1)

然后是模型融合部分所需的第三方库、准备的数据以及训练好的模型：

In [507]:
# 本节新增第三方库
from joblib import dump, load
from sklearn.ensemble import VotingClassifier
from hyperopt import hp, fmin, tpe
from numpy.random import RandomState
from sklearn.model_selection import cross_val_score

In [508]:
class VotingClassifier_threshold(BaseEstimator, ClassifierMixin, TransformerMixin):
    
    def __init__(self, estimators, voting='hard', weights=None, thr=0.5):
        self.estimators = estimators
        self.voting = voting
        self.weights = weights
        self.thr = thr
        
    def fit(self, X, y):
        VC = VotingClassifier(estimators = self.estimators, 
                              voting = self.voting, 
                              weights = self.weights)
        
        VC.fit(X, y)
        self.clf = VC
        
        return self
        
    def predict_proba(self, X):
        if self.voting == 'soft':
            res_proba = self.clf.predict_proba(X)
        else:
            res_proba = None
        return res_proba
    
    def predict(self, X):
        if self.voting == 'soft':
            res = (self.clf.predict_proba(X)[:, 1] >= self.thr) * 1
        else:
            res = self.clf.predict(X)
        return res
    
    def score(self, X, y):
        acc = accuracy_score(self.predict(X), y)
        return acc

In [509]:
# 实例化KFold评估器
kf = KFold(n_splits=5, random_state=12, shuffle=True)

# 重置训练集和测试集的index
X_train_OE = X_train_OE.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

train_part_index_l = []
eval_index_l = []

for train_part_index, eval_index in kf.split(X_train_OE, y_train):
    train_part_index_l.append(train_part_index)
    eval_index_l.append(eval_index)
    
# 训练集特征
X_train1 = X_train_OE.loc[train_part_index_l[0]]
X_train2 = X_train_OE.loc[train_part_index_l[1]]
X_train3 = X_train_OE.loc[train_part_index_l[2]]
X_train4 = X_train_OE.loc[train_part_index_l[3]]
X_train5 = X_train_OE.loc[train_part_index_l[4]]

# 验证集特征
X_eval1 = X_train_OE.loc[eval_index_l[0]]
X_eval2 = X_train_OE.loc[eval_index_l[1]]
X_eval3 = X_train_OE.loc[eval_index_l[2]]
X_eval4 = X_train_OE.loc[eval_index_l[3]]
X_eval5 = X_train_OE.loc[eval_index_l[4]]

# 训练集标签
y_train1 = y_train.loc[train_part_index_l[0]]
y_train2 = y_train.loc[train_part_index_l[1]]
y_train3 = y_train.loc[train_part_index_l[2]]
y_train4 = y_train.loc[train_part_index_l[3]]
y_train5 = y_train.loc[train_part_index_l[4]]

# 验证集标签
y_eval1 = y_train.loc[eval_index_l[0]]
y_eval2 = y_train.loc[eval_index_l[1]]
y_eval3 = y_train.loc[eval_index_l[2]]
y_eval4 = y_train.loc[eval_index_l[3]]
y_eval5 = y_train.loc[eval_index_l[4]]

train_set = [(X_train1, y_train1), 
             (X_train2, y_train2), 
             (X_train3, y_train3), 
             (X_train4, y_train4), 
             (X_train5, y_train5)]

eval_set = [(X_eval1, y_eval1), 
            (X_eval2, y_eval2), 
            (X_eval3, y_eval3), 
            (X_eval4, y_eval4), 
            (X_eval5, y_eval5)]

In [510]:
# 随机森林模型组
grid_RF_1 = load('./models/grid_RF_1.joblib') 
grid_RF_2 = load('./models/grid_RF_2.joblib') 
grid_RF_3 = load('./models/grid_RF_3.joblib') 
grid_RF_4 = load('./models/grid_RF_4.joblib') 
grid_RF_5 = load('./models/grid_RF_5.joblib') 

RF_1 = grid_RF_1.best_estimator_
RF_2 = grid_RF_2.best_estimator_
RF_3 = grid_RF_3.best_estimator_
RF_4 = grid_RF_4.best_estimator_
RF_5 = grid_RF_5.best_estimator_

RF_l = [RF_1, RF_2, RF_3, RF_4, RF_5]

# 决策树模型组
grid_tree_1 = load('./models/grid_tree_1.joblib')
grid_tree_2 = load('./models/grid_tree_2.joblib')
grid_tree_3 = load('./models/grid_tree_3.joblib')
grid_tree_4 = load('./models/grid_tree_4.joblib')
grid_tree_5 = load('./models/grid_tree_5.joblib')

tree_1 = grid_tree_1.best_estimator_
tree_2 = grid_tree_2.best_estimator_
tree_3 = grid_tree_3.best_estimator_
tree_4 = grid_tree_4.best_estimator_
tree_5 = grid_tree_5.best_estimator_

tree_l = [tree_1, tree_2, tree_3, tree_4, tree_5]

# 逻辑回归模型组
grid_lr_1 = load('./models/grid_lr_1.joblib')
grid_lr_2 = load('./models/grid_lr_2.joblib')
grid_lr_3 = load('./models/grid_lr_3.joblib')
grid_lr_4 = load('./models/grid_lr_4.joblib')
grid_lr_5 = load('./models/grid_lr_5.joblib')

lr_1 = grid_lr_1.best_estimator_
lr_2 = grid_lr_2.best_estimator_
lr_3 = grid_lr_3.best_estimator_
lr_4 = grid_lr_4.best_estimator_
lr_5 = grid_lr_5.best_estimator_

lr_l = [lr_1, lr_2, lr_3, lr_4, lr_5]

In [511]:
eval1_predict_proba_RF = pd.Series(RF_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index)
eval2_predict_proba_RF = pd.Series(RF_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index)
eval3_predict_proba_RF = pd.Series(RF_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index)
eval4_predict_proba_RF = pd.Series(RF_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index)
eval5_predict_proba_RF = pd.Series(RF_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index)

eval_predict_proba_RF = pd.concat([eval1_predict_proba_RF, 
                                   eval2_predict_proba_RF, 
                                   eval3_predict_proba_RF, 
                                   eval4_predict_proba_RF, 
                                   eval5_predict_proba_RF]).sort_index()

eval1_predict_proba_tree = pd.Series(tree_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index)
eval2_predict_proba_tree = pd.Series(tree_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index)
eval3_predict_proba_tree = pd.Series(tree_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index)
eval4_predict_proba_tree = pd.Series(tree_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index)
eval5_predict_proba_tree = pd.Series(tree_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index)

eval_predict_proba_tree = pd.concat([eval1_predict_proba_tree, 
                                     eval2_predict_proba_tree, 
                                     eval3_predict_proba_tree, 
                                     eval4_predict_proba_tree, 
                                     eval5_predict_proba_tree]).sort_index()

eval1_predict_proba_lr = pd.Series(lr_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index)
eval2_predict_proba_lr = pd.Series(lr_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index)
eval3_predict_proba_lr = pd.Series(lr_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index)
eval4_predict_proba_lr = pd.Series(lr_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index)
eval5_predict_proba_lr = pd.Series(lr_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index)

eval_predict_proba_lr = pd.concat([eval1_predict_proba_lr, 
                                   eval2_predict_proba_lr, 
                                   eval3_predict_proba_lr, 
                                   eval4_predict_proba_lr, 
                                   eval5_predict_proba_lr]).sort_index()

In [512]:
test_predict_proba_RF = []

for i in range(5):
    test_predict_proba_RF.append(RF_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_RF = np.array(test_predict_proba_RF)
test_predict_proba_RF = test_predict_proba_RF.mean(0)

test_predict_proba_tree = []

for i in range(5):
    test_predict_proba_tree.append(tree_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_tree = np.array(test_predict_proba_tree)
test_predict_proba_tree = test_predict_proba_tree.mean(0)

test_predict_proba_lr = []

for i in range(5):
    test_predict_proba_lr.append(lr_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_lr = np.array(test_predict_proba_lr)
test_predict_proba_lr = test_predict_proba_lr.mean(0)

## <center>Ch.3 模型融合基础方法

- 如何获得更好的融合结果

&emsp;&emsp;不同于模型训练和超参数优化，想要在模型融合过程中获得更好的结果，则必须进行更广泛的尝试，需要多种不同方法的实践、以及实践经验的积累。而在实际学习的过程中，则需要注意前言技术方法的掌握，以及工程化工具的储备。

- Stacking融合优化策略综述

&emsp;&emsp;通过上一小节的学习，我们不难发现，Stacking模型融合和投票法&均值法类似，原理不难，但要获得一个稳定的优化效果却并没有那么简单。并且相比投票法&均值法，Stacking过程采用模型来学习一级评估器的输出结果和标签之间的关系，过拟合的倾向会更加明显。当然，关于Stacking容易过拟合的另一个理解的角度是：由于第一层学习器就已经提取了和标签更有关联度的特征，因此元学习器的学习难度偏弱，元学习器更容易过拟合。      
&emsp;&emsp;此外，在上一小节我们分别尝试了手动Stacking和调用sklearn库实现Stacking融合，从本节开始，我们将重点介绍Stacking模型融合的优化策略。上一小节中我们围绕Stacking最核心的三个方向已经分别进行了尝试，分别是：
- 其一是一、二级学习器优化，包括一级学习器训练方法优化与元学习器优化。这是最基础同时也是最核心的优化策略，其核心目的在于平衡Stacking融合的学习效果与过拟合倾向之间的关系；     
- 其二则是多层Stacking，通过叠加更多层来提高Stacking的学习效果，当然在大多数情况下解决单层Stacking融合的过拟合问题已属实不易，要用好多层Stacking则更是难上加难；      
- 其三则是特征增强。所谓特征增强，指的是一级学习器和元学习器带入不同的特征（或衍生特征的）组合，来提高模型多样性，并最终提升Stacking融合效果。

> 这里需要注意，我们所谓的特征增强更多是实践过程中对某一类方法总结的名称，这类方法在学术上还有另一种叫法：输入属性扰动。属于Stacking多样性增强的一类方法。

> 其实，就这三个优化方向来说，元学习器优化是每个Stacking过程都必须要进行的基础优化策略，而多层Stacking实际应用场景中出现的不多，但偶尔对于某些复杂数据集会有奇效，而真正能够大幅提升Stacking融合效果的方法其实是特征增强。我们知道，投票法&均值法在进行模型融合时，发挥效果的方式类似于Bagging，会更适用于“合而不同”的一组评估器的融合，即只要不同模型输出结果不同，就有可能融合得到一个更好的结果。但Stacking完全不同，根据Stacking的基本原理，该过程其实更适用于一组能够有效学习不同特征的一级评估器的融合。而如何能让一级学习器来各有所偏重的学习不同特征？特征增强肯定是最佳选择。例如我们完全可以给不同模型分配不同特征的衍生特征，从而让不同模型的学习重点各有不同，进而提升Stacking最终效果。此外，在元学习器中添加一些一级学习器中没有充分学习的衍生特征（例如特征重要性偏低的特征），也是能提升Stacking效果的。

&emsp;&emsp;接下来，我们就这三个优化方向、同时结合Stacking融合发挥作用的根本方式来进行更进一步的探讨。本小节将重点探讨Stacking过程中一级学习器的交叉训练策略、元学习器选择以及元学习器优化相关内容。

## 八、Stacking一级学习器交叉训练策略

&emsp;&emsp;我们知道，Stacking强大的学习能力以及显著的过拟合问题，都源于其分层学习的架构特点（这点和神经网络类似）。如何克服过拟合问题、提升Stacking泛化能力，首先就必须要从各层学习器的训练和优化过程入手进行调整。           

### 1.一级学习器的三种不同的交叉训练策略

&emsp;&emsp;首先是一级学习器的训练过程。上一小节提到，作为提升模型融合泛化能力的重要手段，交叉训练已经承了Stacking一级学习器训练过程的“标配”，而在执行一级学习器交叉训练的过程，其实又可以分为三种不同的交叉训练的方式：    

- 方法一：直接带入原始参数评估器进行交叉训练，即每一组模型内部都共用一组模型原始超参数。由于没有进行超参数优化，因此该种方式能够非常快速的训练一组模型，并可以借助sklearn中Stacking评估器来快速执行，该过程可以通过如下方式实现：

In [513]:
# 实例化一级评估器（默认超参数）
logistic = LogisticRegression()
tree = DecisionTreeClassifier()
RF = RandomForestClassifier()

# 实例化Stacking评估器
estimators = [('lr', logistic), ('tree', tree), ('rf', RF)]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

# 在训练集上训练
clf.fit(X_train_OE, y_train)

# 输出训练集和测试集评分
clf.score(X_train_OE, y_train), clf.score(X_test_OE, y_test)

(0.8761832639151836, 0.7853492333901193)

不过需要注意的是，该方式尽管足够简洁，但由于一级学习器没有进行超参数优化，因此最终Stacking的结果表现出了较为明显的过拟合倾向。除非是希望快速验证某些建模过程，否则一般不建议采用该种方式进行一级学习器的训练。

- 方法二：先在全部训练集上训练一级学习器，并进行超参数优化。在确定每个一级学习器的超参数取值后，再进行交叉训练，交叉训练过程只训练每个一级学习器的参数而非超参数。尽管此时每一组模型内部超参数仍然是相同取值，但由于这组超参数毕竟是训练集上训练得到，因此方案二的Stacking结果的泛化能力要强于方案一，具体实现过程如下：

In [514]:
# 实例化一级评估器（默认超参数）
logistic_search = load('./models/logistic_search.joblib') 
tree_model = load('./models/tree_model.joblib') 
RF_0 = load('./models/RF_0.joblib') 

# 实例化Stacking评估器
estimators = [('lr', logistic_search.best_estimator_), ('tree', tree_model), ('rf', RF_0)]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

# 在训练集上训练
clf.fit(X_train_OE, y_train)

# 输出训练集和测试集评分
clf.score(X_train_OE, y_train), clf.score(X_test_OE, y_test)

(0.8273381294964028, 0.787052810902896)

这里三个一级学习器的超参数其实就是在全部训练集上搜索得到的一组最优结果。根据最终Stacking评分能够看出，过拟合倾向得到了明显抑制，但融合结果并没有显著提升。实际上，在大多数情况下，第二种训练策略是能够一定程度抑制Stacking过拟合倾向，同时测试集上结果（即泛化能力）也会略好于方案一。

|train strategy|train_score|test_score|
|:--:|:--:|:--:|
|组内共用默认超参数|0.88337|0.7898|
|组内共用优化后的超参数|0.82733|0.7870|

> 大家可以思考，为何过拟合被抑制但测试集评分却略有下降。

- 方法三：：在交叉训练过程中，每个模型的每个组内训练过程都单独进行超参数搜索，正如Part 4.3.4中进行操作，然后再进行Stacking融合。该策略交叉训练过程会耗费大量的时间，并且于需要先确定几折然后再进行一级学习器的超参数搜索和优化，因此会使得我们一般无法再根据融合结果灵活调整Stacking的CV超参数（调整一次就需要重新训练各组模型、重新进行超参数优化，成本巨大）。并且，由于sklearn较高的封装程度，使得我们无法简单调用sklearn中的评估器来完成该过程，整个过程都需要手动编写代码实现。但是，由于这种策略能够显著提升交叉训练过程中每个模型的泛化能力（更严格的信息隔离）和多样性（超参数多样性），因此在大多数情况下都是能显著提升最终融合结果的。例如我们借助Part 4.3.4中交叉训练结果，在不改变元学习器的情况下，能够得到Stacking融合结果如下：

In [515]:
# 随机森林模型组OOF数据集
eval_predict_proba_RF

0       0.044787
1       0.572187
2       0.161815
3       0.250871
4       0.122533
          ...   
5277    0.082653
5278    0.346562
5279    0.551481
5280    0.049011
5281    0.002783
Length: 5282, dtype: float64

In [516]:
train_stack_oof = pd.DataFrame({'lr_oof': eval_predict_proba_lr, 
                                'tree_oof': eval_predict_proba_tree, 
                                'RF_oof': eval_predict_proba_RF})

train_stack_oof

,lr_oof,tree_oof,RF_oof
0,0.011289,0.037669,0.044787
1,0.542331,0.787986,0.572187
2,0.154121,0.222819,0.161815
3,0.273393,0.259434,0.250871
4,0.158399,0.107345,0.122533
...,...,...,...
5277,0.083575,0.062959,0.082653
5278,0.365228,0.222819,0.346562
5279,0.674365,0.438538,0.551481
5280,0.050536,0.066419,0.049011


In [517]:
test_predict_proba_tree

array([0.04647312, 0.15890043, 0.04647312, ..., 0.15890043, 0.43740054,
       0.1303992 ])

In [518]:
test_stack = pd.DataFrame({'lr_test': test_predict_proba_lr, 
                           'tree_test': test_predict_proba_tree, 
                           'RF_test': test_predict_proba_RF})

test_stack

,lr_test,tree_test,RF_test
0,0.045946,0.046473,0.029220
1,0.233711,0.158900,0.311980
2,0.005192,0.046473,0.016244
3,0.035878,0.046473,0.025769
4,0.056142,0.051573,0.035476
...,...,...,...
1756,0.169545,0.212513,0.193367
1757,0.040112,0.046473,0.048821
1758,0.125325,0.158900,0.145346
1759,0.501991,0.437401,0.530686


In [519]:
lr_final = LogisticRegression().fit(train_stack_oof, y_train)

In [520]:
lr_final.score(train_stack_oof, y_train), lr_final.score(test_stack, y_test)

(0.8178720181749337, 0.7955706984667802)

能够看出，此时Stacking融合结果，不仅过拟合倾向得到了有效抑制，最终结果的泛化能力也得到了显著提升。

|train strategy|train_score|test_score|
|:--:|:--:|:--:|
|组内共用默认超参数|0.88337|0.7898|
|组内共用优化后的超参数|0.82733|0.7870|
|组内单独进行超参数优化|0.81787|0.7955|

&emsp;&emsp;尽管从效果上来说，第三种训练策略优势明显（多数情况下能够有千分位上的效果提升），但在具体实践过程中选取哪种训练策略，还需要根据具体实践情况来决定。当然，在大多数情况下我们主要是围绕第二、三种策略来进行选择，一般来说除非为了快速验证某些结论，否则不会采用第一种方法。      
&emsp;&emsp;而就第二、三种方法的选择来说，如果时间允许，并且追求效果上的极致，建议考虑第三种训练策略，而如果时间有限，第二种训练策略也不失为一种能够保证效果的方案，并且大多数的建模情况下，我们都是要围绕单模进行超参数优化的，因此方案二的实践成本是很低的。实际上，在多数情况下我们看到的Stacking的交叉训练，其实都是采用的方案二，而很多方案二的训练结果也是比较可观的。

> 之前的Kaggle案例公开课的Stacking融合过程就是采用的方案二。

&emsp;&emsp;对于方案二，其实还有一种变种策略，能实现类似方案二的效果：即带入网格搜索评估器而非模型评估器，则可以在交叉训练过程中每次训练都在既定的超参数空间中搜索得到一个最佳超参数组。尽管面对更加复杂的集成算法，我们没法最开始设置一个非常大的超参数搜索空间（否则会导致一次计算的时间过长），但哪怕是一个小范围内的搜索结果，也能够一定程度提升模型效果。这一点我们稍后会进行尝试。

&emsp;&emsp;此外，方案三的实现方法其实也并非一定像Part 4.3.4中那样，需要一个个模型单独来进行超参数优化。在下一小节，我们将借助级联优化的思路，来尝试构建一种更加自动化的流程来高效率的实现方案三。总的来说，课上会同时使用两种方案来进行Stacking融合，并从运行效率和融合结果等多个角度来进行分析和对比，同时完整提供两种方案的实现代码，方便同学们在实践过程中自行选择。

### 2.交叉训练的函数封装

&emsp;&emsp;尽管sklearn的Stacking评估器调用过程足够简洁，但由于其较高的封装程度，很多时候并不利于我们来进行后续更加灵活的优化操作，因此在绝大多数情况下，我们都是手动进行一级评估器的交叉训练。此处我们先将手动交叉训练的相关过程封装为一个函数，方便后续调用：

In [521]:
def train_cross(X_train, y_train, X_test, estimators, n_splits=5, random_state=12):
    """
    Stacking融合过程一级学习器交叉训练函数
    
    :param X_train: 训练集特征
    :param y_train: 训练集标签
    :param X_test: 测试集特征
    :param estimators: 一级学习器，由(名称,评估器)组成的列表
    :param n_splits: 交叉训练折数
    :param random_state: 随机数种子
    
    :return：交叉训练后创建oof训练数据和测试集平均预测结果
    """
    # 重置数据集的index，若数据集index是顺序排列，则此步骤可省略
    X = X_train.reset_index(drop=True)
    y = y_train.reset_index(drop=True)
    
    # 实例化重复交叉验证评估器
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # 创建一级评估器输出的训练集预测结果和测试集预测结果数据集
    # 数据集当前数值暂用零值填充
    m = X.shape[0]
    n = len(estimators)
    m_test = X_test.shape[0]
    
    columns = []
    for estimator in estimators:
        columns.append(estimator[0] + '_oof')
    
    train_oof = pd.DataFrame(np.zeros((m, n)), columns=columns)
    
    columns = []
    for estimator in estimators:
        columns.append(estimator[0] + '_predict')
    
    test_predict = pd.DataFrame(np.zeros((m_test, n)), columns=columns)

    # 执行交叉训练
    for estimator in estimators:
        model = estimator[1]
        oof_colName = estimator[0] + '_oof'
        predict_colName = estimator[0] + '_predict'
        
        for train_part_index, eval_index in kf.split(X, y):
            # 在训练集上训练
            X_train_part = X.loc[train_part_index]
            y_train_part = y.loc[train_part_index]
            model.fit(X_train_part, y_train_part)
            # 在验证集上进行验证
            X_eval_part = X.loc[eval_index]
            # 将验证集上预测结果拼接入oof数据集
            train_oof[oof_colName].loc[eval_index] = model.predict_proba(X_eval_part)[:, 1]
            # 将测试集上预测结果填入predict数据集
            test_predict[predict_colName] += model.predict_proba(X_test)[:, 1] / n_splits
        
    return train_oof, test_predict

&emsp;&emsp;接下来测试函数性能：

In [522]:
logistic_search = load('./models/logistic_search.joblib') 
tree_model = load('./models/tree_model.joblib') 
RF_0 = load('./models/RF_0.joblib') 

estimators = [('lr', logistic_search.best_estimator_), ('tree', tree_model), ('rf', RF_0)]

In [523]:
train_oof, test_predict = train_cross(X_train_OE, y_train, X_test_OE, estimators=estimators)

In [524]:
train_oof

,lr_oof,tree_oof,rf_oof
0,0.010385,0.065380,0.064009
1,0.553209,0.787986,0.623811
2,0.146526,0.222819,0.186379
3,0.291518,0.234201,0.243126
4,0.156131,0.211248,0.158017
...,...,...,...
5277,0.064633,0.062959,0.096975
5278,0.351549,0.222819,0.257227
5279,0.682007,0.503722,0.478084
5280,0.054905,0.066419,0.058595


In [525]:
test_predict

,lr_predict,tree_predict,rf_predict
0,0.035692,0.052015,0.023194
1,0.231090,0.091196,0.307647
2,0.006166,0.052015,0.006359
3,0.029141,0.052015,0.012658
4,0.065028,0.069881,0.049331
...,...,...,...
1756,0.178666,0.223305,0.192447
1757,0.030765,0.052015,0.066831
1758,0.136318,0.091196,0.160348
1759,0.500557,0.484657,0.591940


通过该函数，我们能够快速获得一级评估器交叉训练得到的oof训练数据集，以及测试集上的平均预测结果。有了该结果，我们就能进一步对元学习器进行训练：

In [526]:
clf = LogisticRegression().fit(train_oof, y_train)

In [527]:
clf.score(train_oof, y_train), clf.score(test_predict, y_test)

(0.811056418023476, 0.7893242475865985)

## 九、元学习器的模型选择与超参数优化

&emsp;&emsp;当然，就整个Stacking过程模型训练优化的讨论来说，相比一级学习器的训练和优化，更关键、影响更大的，其实是元学习器的选择与优化。根据上一小节的探索，我们已经发现，Stacking元学习器的训练过程极容易过拟合，究其原因，其实是oof训练数据学习难度较低导致。因此，元学习器的模型选择并不是越复杂越好，在上一小节中，我们得出了以下基本结论：

- 简单模型要比复杂模型效果更好：例如逻辑回归、决策树等模型作为元学习器，就会比随机森林等更加复杂的模型建模效果更好；

- 手动限制模型过拟合往往能得到一个更好的结果：例如将逻辑回归模型中正则化项设置为l1正则化、或者限制决策树模型的结构复杂度，往往能够获得一个更好的建模结果。

接下来，我们以此为基础，进一步讨论关于元学习器的选择和超参数优化策略。

### 1.元学习器模型选择

&emsp;&emsp;关于元学习的模型选择，上一小节我们曾借助Stacking评估器快速验证了逻辑回归作为分类模型效果的优越性，接下来我们快速进行一组更加严谨的实验：通过带入更大范围的模型来验证最佳元学习器的模型选择，同时对其进行超参数优化，试探其效果上限。

- 多组模型的元学习器效果测试

&emsp;&emsp;这里我们首先带入全部常用分类模型进行元学习器的模型训练，并测试最终学习效果：

In [528]:
# 逻辑回归
lr = LogisticRegression().fit(train_oof, y_train)
print('The results of LR-final:')
print('Train-Accuracy: %f, Test-Accuracy: %f' % (lr.score(train_oof, y_train), lr.score(test_predict, y_test)))

# 决策树
tree = DecisionTreeClassifier().fit(train_oof, y_train)
print('The results of tree-final:')
print('Train-Accuracy: %f, Test-Accuracy: %f' % (tree.score(train_oof, y_train), tree.score(test_predict, y_test)))

# KNN最近邻分类器
from sklearn import neighbors
KNN = neighbors.KNeighborsClassifier().fit(train_oof, y_train)
print('The results of KNN-final:')
print('Train-Accuracy: %f, Test-Accuracy: %f' % (KNN.score(train_oof, y_train), KNN.score(test_predict, y_test)))

# SVM支持向量机
from sklearn import svm
SVM = svm.SVC().fit(train_oof, y_train)
print('The results of SVM-final:')
print('Train-Accuracy: %f, Test-Accuracy: %f' % (SVM.score(train_oof, y_train), SVM.score(test_predict, y_test)))

# 朴素贝叶斯/高斯贝叶斯
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(train_oof, y_train)
print('The results of GaussianNB-final:')
print('Train-Accuracy: %f, Test-Accuracy: %f' % (gnb.score(train_oof, y_train), gnb.score(test_predict, y_test)))

# Bagging
from sklearn.ensemble import BaggingClassifier
bagging = BaggingClassifier().fit(train_oof, y_train)
print('The results of Bagging-final:')
print('Train-Accuracy: %f, Test-Accuracy: %f' % (bagging.score(train_oof, y_train), bagging.score(test_predict, y_test)))

# 随机森林
RFC = RandomForestClassifier().fit(train_oof, y_train)
print('The results of RandomForest-final:')
print('Train-Accuracy: %f, Test-Accuracy: %f' % (RFC.score(train_oof, y_train), RFC.score(test_predict, y_test)))

# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
ABC = AdaBoostClassifier().fit(train_oof, y_train)
print('The results of AdaBoost-final:')
print('Train-Accuracy: %f, Test-Accuracy: %f' % (ABC.score(train_oof, y_train), ABC.score(test_predict, y_test)))

# GBDT
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier().fit(train_oof, y_train)
print('The results of GBDT-final:')
print('Train-Accuracy: %f, Test-Accuracy: %f' % (GBC.score(train_oof, y_train), GBC.score(test_predict, y_test)))

# XGB
from xgboost import XGBClassifier
XGB = XGBClassifier().fit(train_oof, y_train)
print('The results of XGB-final:')
print('Train-Accuracy: %f, Test-Accuracy: %f' % (XGB.score(train_oof, y_train), XGB.score(test_predict, y_test)))

The results of LR-final:
Train-Accuracy: 0.811056, Test-Accuracy: 0.789324
The results of tree-final:
Train-Accuracy: 0.999432, Test-Accuracy: 0.716070
The results of KNN-final:
Train-Accuracy: 0.850625, Test-Accuracy: 0.755821
The results of SVM-final:
Train-Accuracy: 0.810867, Test-Accuracy: 0.787621
The results of GaussianNB-final:
Train-Accuracy: 0.797993, Test-Accuracy: 0.781942
The results of Bagging-final:
Train-Accuracy: 0.980689, Test-Accuracy: 0.764338
The results of RandomForest-final:
Train-Accuracy: 0.999432, Test-Accuracy: 0.767178
The results of AdaBoost-final:
Train-Accuracy: 0.812382, Test-Accuracy: 0.791028
The results of GBDT-final:
Train-Accuracy: 0.827149, Test-Accuracy: 0.787621
[00:43:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restor

能够发现，除了KNN存在欠拟合的情况，其模型都存在不同程度过拟合，而就目前这组模型来说，逻辑回归和AdaBoost表现较好。当然，KNN的欠拟合问题可以通过增加最近邻个数来进行验证：

In [529]:
KNN = neighbors.KNeighborsClassifier(n_neighbors=9).fit(train_oof, y_train)
KNN.score(train_oof, y_train), KNN.score(test_predict, y_test)

(0.827906096175691, 0.7705848949460534)

而对于逻辑回归和AdaBoost来说，逻辑回归模型表现较好的原因是本身oof数据集学习难度就不大，外加模型自带抗过拟合的正则化项，因此最后能得到一个还不错的训练结果；而对于AdaBoost而言，则是因为在默认参数情况下，AdaBoost是以最大深度为1的决策树模型为基础模型进行迭代，外加oof数据集的不同特征其实都和标签保持着非常高的关联度，种种原因，该会使得AdaBoost也天然具备一定的抗过拟合特性，从而使得最终获得了一个还不错的建模效果。

&emsp;&emsp;需要注意的是，在元学习器的选择和优化的过程中，早些年的实践，人们往往倾向于“无脑”选择逻辑回归（回归问题选择贝叶斯回归），而近些年，随着实践应用程度的加深，以及越来越多的特征增强的手段出现，以初级Bagging和Boosting结合逻辑回归（或决策树模型）作为基础学习器的元学习器训练策略也逐渐展露头角。这类方法，也将是我们接下来介绍的关于元学习器优化策略的核心。

&emsp;&emsp;当然，原始参数的建模结果可能并不能完全说明问题，接下来我们进一步围绕这些模型，挑选具有代表性的模型进行超参数优化，并观察最终训练成果。

### 2.元学习器的超参数优化

&emsp;&emsp;接下来我们挑选具备代表性的逻辑回归、决策树、Bagging、AdaBoost和随机森林五个模型来进行元学习器的超参数优化。由于本节我们采用的是手动训练一级学习器的方法，因此可以直接在一级学习器输出的oof数据集上完成元学习器的超参数优化，而不用像上一小节那样根据每个模型不同的超参数，对Stacking元学习器进行修改。

&emsp;&emsp;首先是逻辑回归模型，超参数优化过程如下：

In [530]:
# 设置超参数空间
logistic_param = [
    {'thr': np.arange(0.1, 1, 0.1).tolist(), 'penalty': ['l1'], 'C': np.arange(0.1, 1.1, 0.1).tolist(), 'solver': ['saga']}, 
    {'thr': np.arange(0.1, 1, 0.1).tolist(), 'penalty': ['l2'], 'C': np.arange(0.1, 1.1, 0.1).tolist(), 'solver': ['lbfgs', 'newton-cg', 'sag', 'saga']}, 
]

In [531]:
# 实例化相关评估器
logistic_final = logit_threshold(max_iter=int(1e6))
    
# 执行网格搜索
lfg = GridSearchCV(estimator = logistic_final,
                   param_grid = logistic_param,
                   scoring='accuracy',
                   n_jobs = 15).fit(train_oof, y_train)

In [532]:
lfg.best_score_

0.811434591898168

In [533]:
lfg.best_params_

{'C': 1.0, 'penalty': 'l1', 'solver': 'saga', 'thr': 0.5}

In [534]:
lfg.score(train_oof, y_train), lfg.score(test_predict, y_test)

(0.8120030291556228, 0.7898921067575241)

超参数搜索优化后对比结果如下：

|得分|训练集|测试集|
|:--:|:--:|:--:|
|优化前|0.8110|0.7893|
|优化后|0.8120|0.7898|

能够发现，超参数搜索得到了一个更严格控制结构风险的参数组合（l1）正则化项，并且测试集上准确率略有提升。这也说明超参数优化在逻辑回归模型上确实能发挥作用。

&emsp;&emsp;接下来是决策树模型：

In [535]:
# 实例化决策树评估器
tree_final = DecisionTreeClassifier()

tree_param = {'max_depth': np.arange(2, 16, 1).tolist(), 
              'min_samples_split': np.arange(1, 5, 1).tolist(), 
              'min_samples_leaf': np.arange(1, 4, 1).tolist(), 
              'max_leaf_nodes':np.arange(6, 30, 1).tolist()}

# 实例化网格搜索评估器
tfg = GridSearchCV(estimator = tree_final,
                   param_grid = tree_param,
                   n_jobs = 12)

tfg.fit(train_oof, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=12,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                       14, 15],
                         'max_leaf_nodes': [6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                                            16, 17, 18, 19, 20, 21, 22, 23, 24,
                                            25, 26, 27, 28, 29],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [1, 2, 3, 4]})

In [536]:
tfg.best_params_

{'max_depth': 3,
 'max_leaf_nodes': 7,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [537]:
tfg.best_score_

0.8072704337605

In [538]:
tfg.score(train_oof, y_train), tfg.score(test_predict, y_test)

(0.8087845513063233, 0.7830777967064169)

|得分|训练集|测试集|
|:--:|:--:|:--:|
|优化前|0.9994|0.7177|
|优化后|0.8087|0.7830|

能够发现，超参数优化也同样能提升决策树模型作为元学习器的预测效果并抑制过拟合。不过并未“反超”逻辑回归模型效果。

&emsp;&emsp;接下来是Bagging和AdaBoost，作为较为基础的集成算法，这里我们不仅需要对其进行超参数优化，而且也要尝试在更换不同基础评估器的情况下最终建模效果。首先是Bagging，在不修改baseEstimator的情况下搜索结果如下：

In [539]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "n_estimators": range(10, 21), 
    "max_samples": np.arange(0.1, 1.1, 0.1).tolist()}

# 实例化模型与评估器
bagging_final = BaggingClassifier(DecisionTreeClassifier())
BG = GridSearchCV(bagging_final, parameter_space, n_jobs=15)

# 模型训练
BG.fit(train_oof, y_train)

print(time.time()-start)

4.832921981811523


In [540]:
BG.best_params_

{'max_samples': 0.1, 'n_estimators': 18}

In [541]:
BG.best_score_

0.8057567156904906

In [542]:
BG.score(train_oof, y_train), BG.score(test_predict, y_test)

(0.8322605073835668, 0.7751277683134583)

|得分|训练集|测试集|
|:--:|:--:|:--:|
|优化前|0.9795|0.7683|
|优化后|0.8243|0.7722|

能够发现，相比原始Bagging，效果还是有所提升的。当然，根据AdaBoost起作用的原因分析，我们能够看到基础学习器本身的特性其实是会影响到集成算法的预测结果的（尤其是Bagging和AdaBoost这类较为基础的集成学习算法），因此我们尝试将基础评估器改为超参数优化后的决策树（上述过程是带入了原始超参数的决策树作为base estimator）、原始参数下的逻辑回归、以及超参数优化后的逻辑回归模型进行效果测试：

In [543]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "n_estimators": range(10, 21), 
    "max_samples": np.arange(0.1, 1.1, 0.1).tolist()}

# 实例化模型与评估器
bagging_final = BaggingClassifier(DecisionTreeClassifier(max_depth=3,
                                                         max_leaf_nodes=7,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2))
BG = GridSearchCV(bagging_final, parameter_space, n_jobs=15)

# 模型训练
BG.fit(train_oof, y_train)

print(time.time()-start)

1.613081932067871


In [544]:
BG.best_params_

{'max_samples': 0.7000000000000001, 'n_estimators': 17}

In [545]:
BG.best_score_

0.812381382414495

In [546]:
BG.score(train_oof, y_train), BG.score(test_predict, y_test)

(0.8146535403256342, 0.7881885292447472)

能够发现，带入超参数优化后的决策树模型后，Bagging作为元学习器的效果有了更进一步的提升：

|基础评估器|训练集|测试集|
|:--:|:--:|:--:|
|未优化的决策树|0.8243|0.7722|
|优化后的决策树|0.8120|0.7881|

然后带入原始超参数的逻辑回归模型：

In [547]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "n_estimators": range(10, 21), 
    "max_samples": np.arange(0.1, 1.1, 0.1).tolist(),
    'max_features':np.arange(0.1, 1.1, 0.1).tolist()}

# 实例化模型与评估器
bagging_final = BaggingClassifier(LogisticRegression())
BG = GridSearchCV(bagging_final, parameter_space, n_jobs=15)

# 模型训练
BG.fit(train_oof, y_train)

print(time.time()-start)

30.365494966506958


In [548]:
BG.best_params_

{'max_features': 0.9, 'max_samples': 0.9, 'n_estimators': 11}

In [549]:
BG.best_score_

0.8129495642326768

In [550]:
BG.score(train_oof, y_train), BG.score(test_predict, y_test)

(0.8102991291177585, 0.7836456558773425)

能够发现，该结果价于单独逻辑回归模型作为元学习器的效果，但由于原始Bagging模型效果：

|基础评估器|训练集|测试集|
|:--:|:--:|:--:|
|未优化的决策树|0.8243|0.7722|
|优化后的决策树|0.8120|0.7915|
|未优化的逻辑回归|0.8102|0.7864|

接下来带入超参数优化后的逻辑回归模型：

In [551]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "n_estimators": range(10, 21), 
    "max_samples": np.arange(0.1, 1.1, 0.1).tolist(),
    'max_features':np.arange(0.1, 1.1, 0.1).tolist()}

# 实例化模型与评估器
bagging_final = BaggingClassifier(LogisticRegression(penalty='l1',
                                                     solver='saga'))
BG = GridSearchCV(bagging_final, parameter_space, n_jobs=15)

# 模型训练
BG.fit(train_oof, y_train)

print(time.time()-start)

42.1256947517395


In [552]:
BG.best_params_

{'max_features': 0.9, 'max_samples': 0.6, 'n_estimators': 19}

In [553]:
BG.best_score_

0.8137062440870387

In [554]:
BG.score(train_oof, y_train), BG.score(test_predict, y_test)

(0.8121923513820523, 0.7887563884156729)

能够发现，效果有更进一步的提升：

|基础评估器|训练集|测试集|
|:--:|:--:|:--:|
|未优化的决策树|0.8243|0.7722|
|优化后的决策树|0.8120|0.7915|
|未优化的逻辑回归|0.8102|0.7864|
|优化后的逻辑回归|0.8106|0.7881|

据此我们能够得出两个基本结论，其一，对基础评估器进行超参数优化，尤其是对其进行一定程度过拟合限制，是能够提升Bagging最终效果的；其二，在部分情况下，Bagging搭配超参数优化的基础评估器，有可能得到比单独基础评估器作为元学习器更好的结果。而这两种方法：简单模型的超参数优化、以及将优化后的简单模型进行Bagging集成并进一步进行超参数搜索，都是我们在Stacking过程中非常值得进行尝试的策略。

> 切记模型融合开篇介绍的观点，要保证模型融合的效果，则不仅需要否采用复杂前沿的方法，更需要广泛的尝试可能潜在的有价值的方案，更多的尝试往往才是保障最终融合效果的根本方法。

&emsp;&emsp;最后是AdaBoost，相比之下由于Boosting特殊的流程，并不能像Bagging一样随意修改基础学习器（Boosting类算法降低基础学习器的学习能力，将很大程度影响最终模型训练结果），因此作为元学习器，可以提升的空间较为有限，在大多数情况下，我们只能对其进行超参数优化：

In [555]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "n_estimators": range(10, 101), 
    "learning_rate": np.arange(0.01, 0.55, 0.05).tolist(),
    'algorithm':['SAMME.R', 'SAMME']}

# 实例化模型与评估器
AB_final = AdaBoostClassifier()
abg = GridSearchCV(AB_final, parameter_space, n_jobs=15)

# 模型训练
abg.fit(train_oof, y_train)

print(time.time()-start)

90.56694054603577


In [556]:
abg.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 0.11, 'n_estimators': 33}

In [557]:
abg.score(train_oof, y_train), abg.score(test_predict, y_test)

(0.8114350624763347, 0.787052810902896)

能够看出结果其实并没有提升。而如果我们将基础模型换为逻辑回归模型，模型结果也不会有好转：

In [558]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "n_estimators": range(10, 51), 
    "learning_rate": np.arange(0.01, 0.51, 0.05).tolist(),
    'algorithm':['SAMME.R', 'SAMME']}

# 实例化模型与评估器
AB_final = AdaBoostClassifier(LogisticRegression())
abg = GridSearchCV(AB_final, parameter_space, n_jobs=15)

# 模型训练
abg.fit(train_oof, y_train)

print(time.time()-start)

30.944231748580933


In [559]:
abg.best_params_

{'algorithm': 'SAMME', 'learning_rate': 0.26, 'n_estimators': 29}

In [560]:
abg.score(train_oof, y_train), abg.score(test_predict, y_test)

(0.8076486179477471, 0.7864849517319704)

而如果我们希望通过提升基础模型的抗过拟合特性（即提升基础模型的结构风险惩罚力度），则Boosting类算法则因为缺乏学习能力而无法得到有效建模结果：

In [568]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "n_estimators": range(1, 31), 
    "learning_rate": np.arange(0.01, 0.51, 0.05).tolist(),
    'algorithm':['SAMME.R', 'SAMME']}

# 实例化模型与评估器
AB_final = AdaBoostClassifier(LogisticRegression(penalty='l1',
                                                     solver='saga'))
abg = GridSearchCV(AB_final, parameter_space, n_jobs=15)

# 模型训练
abg.fit(train_oof, y_train)

print(time.time()-start)

8.684714317321777


In [569]:
abg.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 0.01, 'n_estimators': 1}

In [570]:
abg.score(train_oof, y_train), abg.score(test_predict, y_test)

(0.7385460053010223, 0.7228847245883021)

&emsp;&emsp;其实不仅仅是AdaBoost，很多更加复杂的集成学习算法经过超参数优化后也并不会有非常显著的效果提升。以随机森林为例，当我们对其进行超参数优化后，并不会获得一个比逻辑回归或者逻辑回归Bagging的更好的结果：

In [564]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "min_samples_leaf": range(2, 6), 
    "min_samples_split": range(1, 6),
    "max_depth": range(5, 8),
    "max_leaf_nodes": [None] + list(range(20, 25)), 
    "n_estimators": range(6, 11), 
    "max_samples":[None, 0.54, 0.55, 0.56]}

# 实例化模型与评估器
RF_final = RandomForestClassifier(random_state=12)
rfg = GridSearchCV(RF_final, parameter_space, n_jobs=15)

# 模型训练
rfg.fit(train_oof, y_train)

print(time.time()-start)

41.03087377548218


In [565]:
rfg.best_params_

{'max_depth': 6,
 'max_leaf_nodes': None,
 'max_samples': 0.55,
 'min_samples_leaf': 3,
 'min_samples_split': 2,
 'n_estimators': 8}

In [566]:
rfg.score(train_oof, y_train), rfg.score(test_predict, y_test)

(0.8188186293070806, 0.7853492333901193)

当然，除了随机森林，其他复杂的集成算法其实也会出现类似情况。同学们感兴趣的话可以课后自行尝试。

&emsp;&emsp;经过上述一系列尝试，我们能获得以下结论：

- 元学习器的超参数优化是能够提升元学习器的泛化能力的，在大多数情况下我们应该尽可能的对其进行超参数优化；

- 除了元学习器应该尽可能选择抗过拟合较高的简单模型外，有效的元学习器（经过超参数优化后的元学习器）+Bagging有时也能一定的提升元学习器学习能力；

需要注意的是，这些基本结论不仅适用于当前数据集，同时也是大多数实践经验总结出来的结果。甚至，由于本案例数据集的特殊性，很多在大多数情况下有显著提升效果的方法（例如使用超参数优化后的逻辑回归作为基础评估器来进行Bagging），在当前数据集上并没有明显的显现其作用。

### 3.元学习器的训练策略

&emsp;&emsp;在得到了上述一系列结论后，Stacking过程中元学习器的训练策略就非常清晰了。总的来说可以分为两个阶段：

- 选取逻辑回归、决策树等模型作为元学习器的备选模型，对其进行训练和超参数优化；

- 以超参数优化后的逻辑回归模型或决策树模型等模型作为基础模型进行Bagging过程，并对Bagging评估器进行超参数优化；

最终，在两个阶段训练得到的若干模型中挑选最好的元学习器输出最终预测结果，当然如果是在竞赛中，也可以多次提交不同元学习器的预测结果，来获得一个当前元学习器中最高的排名。

> 如果是回归问题，则基础分类器则更多考虑贝叶斯回归、Lasso和岭回归，第二阶段的Bagging过程类似。

&emsp;&emsp;不过需要注意的是，其实在单独的基础分类器作为元学习器和Bagging作为元学习器二者之间，还存在一种中间状态，即有些情况下我们并不会直接对基础分类器进行Bagging，而是借助sklearn中交叉验证过程，对其进行多次训练（一般是进行5-10次训练），然后取其平均预测结果作为最终预测结果，以实现类似Bagging但弱于Bagging的过程。该过程也被称为元学习器的交叉训练，只不过元学习器的交叉训练的目标并不是为了得到OOF数据集，而是借助交叉训练实现类似Boostrap的过程，来提升元学习器的泛化能力。具体执行流程如下：

In [571]:
from sklearn.model_selection import RepeatedKFold

In [597]:
res = np.zeros(test_predict.shape[0])

folds = RepeatedKFold(n_splits=5, n_repeats=2)

for trn_idx, val_idx in folds.split(train_oof, y_train):
    lr = LogisticRegression(penalty='l1',solver='saga')
    lr.fit(train_oof.loc[trn_idx], y_train.loc[trn_idx])
    res += lr.predict_proba(test_predict)[:, 1] / 10

In [598]:
accuracy_score((res >= 0.5) * 1, y_test)

0.7898921067575241

In [599]:
res = np.zeros(test_predict.shape[0])

folds = RepeatedKFold(n_splits=5, n_repeats=2)

for trn_idx, val_idx in folds.split(train_oof, y_train):
    tree = DecisionTreeClassifier(max_depth=3,
                                  max_leaf_nodes=7,
                                  min_samples_leaf=1,
                                  min_samples_split=2)
    tree.fit(train_oof.loc[trn_idx], y_train.loc[trn_idx])
    res += tree.predict_proba(test_predict)[:, 1] / 10

In [600]:
accuracy_score((res >= 0.5) * 1, y_test)

0.7887563884156729

其中RepeatedKFold就是简单的重复kFold过程：

In [601]:
from sklearn.model_selection import RepeatedKFold

In [602]:
# 其中n_repeats用于控制重复的次数
folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=12)

In [603]:
trn_idx_l = []

for trn_idx, val_idx in folds.split(train_oof, y_train):
    trn_idx_l.append(trn_idx)

In [604]:
trn_idx_l

[array([   1,    2,    5, ..., 5279, 5280, 5281]),
 array([   0,    1,    2, ..., 5279, 5280, 5281]),
 array([   0,    1,    2, ..., 5279, 5280, 5281]),
 array([   0,    1,    3, ..., 5275, 5277, 5281]),
 array([   0,    2,    3, ..., 5278, 5279, 5280]),
 array([   0,    1,    2, ..., 5278, 5280, 5281]),
 array([   0,    1,    2, ..., 5278, 5279, 5281]),
 array([   0,    3,    5, ..., 5279, 5280, 5281]),
 array([   0,    1,    2, ..., 5279, 5280, 5281]),
 array([   1,    2,    3, ..., 5277, 5279, 5280])]

这里需要注意，尽管是重复执行kFold，但划分的随机过程却不是重复的，这里能够看到，在两轮进行五折划分时，每轮的第一次划分结果都不相同：

In [605]:
trn_idx_l[0]

array([   1,    2,    5, ..., 5279, 5280, 5281])

In [606]:
trn_idx_l[5]

array([   0,    1,    2, ..., 5278, 5280, 5281])

而当我们设置n_repeats=1时，其实就是简单的进行五折数据集划分，效果相当于KFold(n_splits=5)

In [607]:
folds = RepeatedKFold(n_splits=5, n_repeats=1, random_state=12)
for trn_idx, val_idx in folds.split(X_train_OE):
    print(trn_idx)

[   1    2    5 ... 5279 5280 5281]
[   0    1    2 ... 5279 5280 5281]
[   0    1    2 ... 5279 5280 5281]
[   0    1    3 ... 5275 5277 5281]
[   0    2    3 ... 5278 5279 5280]


In [608]:
folds = KFold(n_splits=5, random_state=12, shuffle=True)
for trn_idx, val_idx in folds.split(X_train_OE):
    print(trn_idx)

[   1    2    5 ... 5279 5280 5281]
[   0    1    2 ... 5279 5280 5281]
[   0    1    2 ... 5279 5280 5281]
[   0    1    3 ... 5275 5277 5281]
[   0    2    3 ... 5278 5279 5280]


&emsp;&emsp;当然，有的时候我们也可以考虑在多轮计算的时候直接带入网格搜索评估器而不是确定超参数的模型评估器，以提升每次训练的模型效果。当然，在某些情况下是能够提升最终效果的：

In [614]:
res = np.zeros(test_predict.shape[0])

folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=12)

for trn_idx, val_idx in folds.split(train_oof, y_train):
    lfg = GridSearchCV(estimator = logit_threshold(max_iter=int(1e6)),
                       param_grid = logistic_param,
                       scoring='accuracy',
                       n_jobs = 15)
    lfg.fit(train_oof.loc[trn_idx], y_train.loc[trn_idx])
    res += lfg.predict_proba(test_predict)[:, 1] / 10
    
accuracy_score((res >= 0.5) * 1, y_test)

0.7887563884156729

In [616]:
res = np.zeros(test_predict.shape[0])

folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=12)

for trn_idx, val_idx in folds.split(train_oof, y_train):
    tfg = GridSearchCV(estimator = DecisionTreeClassifier(),
                       param_grid = tree_param,
                       n_jobs = 12)
    tfg.fit(train_oof.loc[trn_idx], y_train.loc[trn_idx])
    res += tfg.predict_proba(test_predict)[:, 1] / 10

print(accuracy_score((res >= 0.5) * 1, y_test))

0.7876206700738216


至此，我们就完整介绍了关于元学习器训练流程。这里需要注意，元学习器单次训练成本较低，可以多次尝试不同方法，并从中挑选最优结果。

### 4.元学习器的训练实战

&emsp;&emsp;最后，让我们在严格信息隔离的交叉训练的oof数据集上，按照我们之前的流程来进行元学习器的完整训练过程，并尝试能否获得一个更好的测试集预测结果。此前最好的结果是Part 4.3.4中获得0.7972。

- Step 1.元学习器的单模优化训练

&emsp;&emsp;首先是逻辑回归和决策树单模训练与超参数优化过程：

In [618]:
# 设置超参数空间
logistic_param = [
    {'thr': np.arange(0.1, 1, 0.1).tolist(), 'penalty': ['l1'], 'C': np.arange(0.1, 1.1, 0.1).tolist(), 'solver': ['saga']}, 
    {'thr': np.arange(0.1, 1, 0.1).tolist(), 'penalty': ['l2'], 'C': np.arange(0.1, 1.1, 0.1).tolist(), 'solver': ['lbfgs', 'newton-cg', 'sag', 'saga']}, 
]

# 实例化相关评估器
logistic_final = logit_threshold(max_iter=int(1e6))
    
# 执行网格搜索
lfg = GridSearchCV(estimator = logistic_final,
                   param_grid = logistic_param,
                   scoring='accuracy',
                   n_jobs = 15).fit(train_stack_oof, y_train)

lfg.score(train_stack_oof, y_train), lfg.score(test_stack, y_test)

(0.8186293070806513, 0.7967064168086314)

In [621]:
lfg.best_params_

{'C': 0.30000000000000004, 'penalty': 'l1', 'solver': 'saga', 'thr': 0.5}

&emsp;&emsp;接下来是决策树模型：

In [619]:
# 实例化决策树评估器
tree_final = DecisionTreeClassifier()

tree_param = {'max_depth': np.arange(2, 16, 1).tolist(), 
              'min_samples_split': np.arange(1, 5, 1).tolist(), 
              'min_samples_leaf': np.arange(1, 4, 1).tolist(), 
              'max_leaf_nodes':np.arange(6, 30, 1).tolist()}

# 实例化网格搜索评估器
tfg = GridSearchCV(estimator = tree_final,
                   param_grid = tree_param,
                   n_jobs = 12).fit(train_stack_oof, y_train)

tfg.score(train_stack_oof, y_train), tfg.score(test_stack, y_test)

(0.8188186293070806, 0.7904599659284497)

In [620]:
tfg.best_params_

{'max_depth': 3,
 'max_leaf_nodes': 7,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

单模元学习器训练结果如下：

|得分|训练集|测试集|
|:--:|:--:|:--:|
|LR单模|0.8186|0.7967|
|Tree单模|0.8188|0.7904|

其中，LR单模的结果已超过融合前单独模型的最好成绩0.7955。

- Step 2.单模的交叉训练过程

In [627]:
# 逻辑回归交叉训练
res = np.zeros(test_predict.shape[0])

folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=12)

for trn_idx, val_idx in folds.split(train_stack_oof, y_train):
    lfg = GridSearchCV(estimator = logit_threshold(max_iter=int(1e6)),
                       param_grid = logistic_param,
                       scoring='accuracy',
                       n_jobs = 15)
    lfg.fit(train_stack_oof.loc[trn_idx], y_train.loc[trn_idx])
    res += lfg.predict_proba(test_stack)[:, 1] / 10
    
print(accuracy_score((res >= 0.5) * 1, y_test))

0.7978421351504826


In [626]:
# 决策树交叉训练过程
res = np.zeros(test_predict.shape[0])

folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=12)

for trn_idx, val_idx in folds.split(train_stack_oof, y_train):
    tfg = GridSearchCV(estimator = DecisionTreeClassifier(),
                       param_grid = tree_param,
                       n_jobs = 12)
    tfg.fit(train_stack_oof.loc[trn_idx], y_train.loc[trn_idx])
    res += tfg.predict_proba(test_stack)[:, 1] / 10

print(accuracy_score((res >= 0.5) * 1, y_test))

0.7904599659284497


|得分|训练集|测试集|
|:--:|:--:|:--:|
|LR单模|0.8186|0.7967|
|Tree单模|0.8188|0.7904|
|LR交叉训练|-|0.7978|
|Tree交叉训练|-|0.7904|

能够发现，逻辑回归的交叉训练结果已超过此前的最好成绩0.7972。

- Step 3.优化后模型的Bagging过程

&emsp;&emsp;最后，尝试带入经过超参数优化的基础评估器到Bagging中来进行训练：

In [624]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "n_estimators": range(10, 21), 
    "max_samples": np.arange(0.1, 1.1, 0.1).tolist(),
    'max_features':np.arange(0.1, 1.1, 0.1).tolist()}

# 实例化模型与评估器
bagging_final = BaggingClassifier(LogisticRegression(C=0.3, 
                                                     penalty='l1',
                                                     solver='saga'))
BG = GridSearchCV(bagging_final, parameter_space, n_jobs=15).fit(train_stack_oof, y_train)

BG.score(train_stack_oof, y_train), BG.score(test_stack, y_test)

(0.8186293070806513, 0.7967064168086314)

In [622]:
# 设置超参数空间
parameter_space = {
    "n_estimators": range(10, 21), 
    "max_samples": np.arange(0.1, 1.1, 0.1).tolist()}

# 实例化模型与评估器
bagging_final = BaggingClassifier(DecisionTreeClassifier(max_depth=3,
                                                         max_leaf_nodes=7,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2))
BG = GridSearchCV(bagging_final, parameter_space, n_jobs=15).fit(train_stack_oof, y_train)

BG.score(train_stack_oof, y_train), BG.score(test_stack, y_test)

(0.8203332071185158, 0.8001135718341851)

能够发现效果有了更进一步的提升，最终决策树模型的Bagging过程得分超过了此前的最高得分，准确率首次突破了80%大关。

|得分|训练集|测试集|
|:--:|:--:|:--:|
|LR单模|0.8186|0.7967|
|Tree单模|0.8188|0.7904|
|LR交叉训练|-|0.7978|
|Tree交叉训练|-|0.7904|
|LR+Bagging|0.8186|0.7967|
|Tree+Bagging|0.8203|0.8001|

至此，我们就完整介绍了Stacking模型融合过程中一级学习器和元学习器的训练及优化流程。在大多数情况下，这些都是我们在执行模型融合过程中有效且通用的流程，在大型竞赛中，若能顺利执行到这一步，基本可以达到复赛得分。